# ⚖️ ARCCS: Automated Regulatory Compliance Classification System

**ARCCS** is a multi-agent system for regulation-level compliance assessment consisting of two coordinated modules:

1. **RPEM** (Regulatory Processing and Extraction Module) - Extracts structured regulations from legal documents
2. **CCM** (Compliance Classification Module) - Classifies compliance and detects violations

The system produces interpretable compliance outputs with:
- Regulation-level labels (Compliant / Not Compliant / Insufficient Information / Human Required)
- Confidence scores
- Structured explanations justifying each assessment


![image](pipeline.png)
---

### Prerequisites
- OpenAI API key
- PDF of the regulation document (e.g., GDPR)
- Terms of Service text file to analyze

## Step 0: Setup & Configuration

In [1]:
# Auto-reload modules for development
%reload_ext autoreload
%autoreload 2

import os
import json

print("✅ Environment ready!")
print("\n📦 ARCCS Modules:")
print("   • RPEM - Regulatory Processing and Extraction Module")
print("   • CCM  - Compliance Classification Module")

✅ Environment ready!

📦 ARCCS Modules:
   • RPEM - Regulatory Processing and Extraction Module
   • CCM  - Compliance Classification Module


In [ ]:
# Configure OpenAI API
import openai

# Set your OpenAI API key here
openai.api_key = "YOUR_OPENAI_API_KEY" # Replace with your actual key

# Verify API key is set
if openai.api_key == "YOUR_OPENAI_API_KEY":
    print("⚠️  WARNING: Please set your actual OpenAI API key!")
else:
    print("✅ OpenAI API key configured")

✅ OpenAI API key configured


---

## Step 1: RPEM - Regulatory Processing and Extraction Module

The **RPEM** module processes legal documents and extracts structured regulation data.

### 1.1 Load the Regulation Document

In [ ]:
from RPEM import load_pdf_document, elements_to_markdown, split_into_sections

# Path to your regulation PDF
REGULATION_PDF_PATH = "./Example Data/Regulations/GDPR.pdf"

# Load and parse the PDF
elements = load_pdf_document(REGULATION_PDF_PATH)

# Convert to markdown
markdown_text = elements_to_markdown(elements)
print(f"📝 Converted to markdown: {len(markdown_text):,} characters")

# Split into sections
sections = split_into_sections(markdown_text)
print(f"📑 Split into {len(sections)} sections")

# Preview first few sections
print(f"\n📋 Sample sections:")
for i, section in enumerate(sections[:5]):
    print(f"   {i+1}. {section['title'][:60]}")

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


📄 Loading document: /Users/giorgosfilandrianos/Desktop/Projects/Compliance Checking/gdpr.pdf


The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


✅ Loaded 1637 elements
📝 Converted to markdown: 356,653 characters
📑 Split into 241 sections

📋 Sample sections:
   1. ## REGULATIONS
   2. ## of 27 April 2016
   3. ## General provisions
   4. ## Subject-matter and objectives
   5. ## Article 2


### 1.2 Extract Regulations Using RPEM (Regulatory Processing and Extraction Module)

Now RPEM will analyze each section and extract structured regulation data.

In [4]:
from RPEM import process_all_sections, collect_all_regulations

# Process all sections with AI
analysis_results = process_all_sections(sections, model="gpt-5.2", verbose=True)

# Collect all extracted regulations
all_regulations = collect_all_regulations(analysis_results)

print(f"\n{'='*60}")
print(f"📋 RPEM EXTRACTION COMPLETE")
print(f"{'='*60}")
print(f"   Sections analyzed: {len(sections)}")
print(f"   Sections with regulations: {sum(1 for r in analysis_results if r.get('contains_regulation'))}")
print(f"   Total regulations extracted: {len(all_regulations)}")


🔍 RPEM: Processing 241 sections

📖 [1/241] ## REGULATIONS
   ⚪ No regulations found
📖 [2/241] ## of 27 April 2016
   ⚪ No regulations found
📖 [2/241] ## of 27 April 2016
   ⚪ No regulations found
📖 [3/241] ## General provisions
   ⚪ No regulations found
📖 [3/241] ## General provisions
   ⚪ No regulations found
📖 [4/241] ## Subject-matter and objectives
   ⚪ No regulations found
📖 [4/241] ## Subject-matter and objectives
   ⚪ No regulations found
📖 [5/241] ## Article 2
   ⚪ No regulations found
📖 [5/241] ## Article 2
   ⚪ No regulations found
📖 [6/241] ## Material scope
   ⚪ No regulations found
📖 [6/241] ## Material scope
   ✅ Found 3 regulation(s)
📖 [7/241] ## Article 3
   ✅ Found 3 regulation(s)
📖 [7/241] ## Article 3
   ⚪ No regulations found
📖 [8/241] ## Territorial scope
   ⚪ No regulations found
📖 [8/241] ## Territorial scope
   ✅ Found 1 regulation(s)
📖 [9/241] ## Article 4
   ✅ Found 1 regulation(s)
📖 [9/241] ## Article 4
   ⚪ No regulations found
📖 [10/241] ## Definitions
   

### 1.3 Quality Filtering

Filter regulations by completeness and quality.

In [5]:
from RPEM import filter_regulations_by_quality

# Filter regulations with minimum quality score
filtered = filter_regulations_by_quality(all_regulations, min_score=40, verbose=True)

# Get high-quality regulations
kept_regulations = filtered["kept"]
review_regulations = filtered["review"]

print(f"\n✅ Using {len(kept_regulations)} high-quality regulations for compliance checking")


🔍 RPEM: Quality Filtering
   Minimum score: 40
   Regulations to analyze: 103

📊 FILTERING RESULTS
   🟢 KEPT (score ≥ 70):      100
   🟡 REVIEW (score ≥ 40):      3
   🔴 DISCARDED (score < 40):   0

✅ Using 100 high-quality regulations for compliance checking


### 1.4 Save Extracted Regulations

In [6]:
# Save the extracted regulations
output_data = {
    "document_analysis": {
        "source_file": REGULATION_PDF_PATH,
        "total_sections": len(sections),
        "total_regulations": len(all_regulations),
        "filtered_regulations": len(kept_regulations)
    },
    "regulations": kept_regulations
}

with open("extracted_regulations.json", "w", encoding="utf-8") as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"💾 Saved {len(kept_regulations)} regulations to 'extracted_regulations.json'")

💾 Saved 100 regulations to 'extracted_regulations.json'


---

## Step 2: Load Document for Compliance Check

Load the Terms of Service document to analyze.

In [17]:
# Path to your Terms of Service document
TERMS_FILE_PATH = "./Example Data/Terms of Use/WhatsApp.txt"

print("📄 Loading Terms of Service document...")

with open(TERMS_FILE_PATH, "r", encoding="utf-8") as f:
    terms_text = f.read()

print(f"✅ Document loaded!")
print(f"   File: {os.path.basename(TERMS_FILE_PATH)}")
print(f"   Characters: {len(terms_text):,}")

print(f"\n📖 Preview (first 500 chars):")
print("-" * 50)
print(terms_text[:500])
print("-" * 50)

📄 Loading Terms of Service document...
✅ Document loaded!
   File: WhatsApp.txt
   Characters: 325,772

📖 Preview (first 500 chars):
--------------------------------------------------
Skip to content
WhatsApp Main Page

Features
Privacy
Help Center
Blog
For Business
Apps
Log in
Download
You are currently viewing an archived version of the Terms of Service. View the current version
or all past versions.

Effective Date: January 4, 2021 (archived versions)

WhatsApp Terms Of Service
Table of Contents

About Our Services
Privacy Policy And User Data
Acceptable Use Of Our Services
Third-Party Services
Licenses
Reporting Third-Party Copyright, Trademark, And Other Intellectual Pro
--------------------------------------------------


---

## Step 3: CCM - Compliance Classification Module

The **CCM** module checks the document against each regulation.

### How CCM Works:
1. For each regulation, CCM searches for **CONTRADICTIONS** in the document
2. A **CONTRADICTION** = document says X but regulation requires Y
3. If no contradiction → **COMPLIANT** ✅
4. If contradiction found → **NON_COMPLIANT** ❌

In [18]:
from CCM import check_all_regulations, generate_compliance_report, print_detailed_report

# Select regulations to check
regulations_to_check = kept_regulations[:20]  # First 20 regulations

print(f"🔍 CCM: COMPLIANCE CLASSIFICATION")
print(f"{'='*60}")
print(f"   Document: {os.path.basename(TERMS_FILE_PATH)}")
print(f"   Regulations to check: {len(regulations_to_check)}")
print(f"{'='*60}")

# Run compliance check
results = check_all_regulations(
    regulations=regulations_to_check,
    proposal_chunk=terms_text,
    model="gpt-5.2",
    verbose=True
)

print(f"\n✅ Compliance check complete!")

🔍 CCM: COMPLIANCE CLASSIFICATION
   Document: WhatsApp.txt
   Regulations to check: 20

🔍 ARCCS - CCM: Compliance Classification Module
Checking 20 regulations...

⚖️ [1/20] General Data Protection Regulation (GDPR) – Material sc...
   ✅ COMPLIANT
⚖️ [2/20] Regulation (EC) No 45/2001 (EU institutions personal da...
   ✅ COMPLIANT
⚖️ [3/20] Directive 2000/31/EC (E-Commerce Directive) – Intermedi...
   ✅ COMPLIANT
⚖️ [4/20] Regulation (EU) 2016/679 (General Data Protection Regul...
   ✅ COMPLIANT
⚖️ [5/20] Regulation (EU) 2016/679 (General Data Protection Regul...
   ✅ COMPLIANT
⚖️ [6/20] Regulation (EU) 2016/679 (General Data Protection Regul...
   ❌ NON_COMPLIANT - Contradiction found!
      → GDPR Art. 5(1)(d) requires personal data to be accurate and kept up to date, and...
⚖️ [7/20] Directive (EU) 2015/1535 of the European Parliament and...
   ✅ COMPLIANT
⚖️ [8/20] Regulation (EU) 2016/679 (General Data Protection Regul...
   ✅ COMPLIANT
⚖️ [9/20] Regulation (EU) 2016/679 (General D

In [19]:
# Generate and display the compliance report
report = generate_compliance_report(results)
print_detailed_report(report)


📊 ARCCS COMPLIANCE REPORT

❌ NON-COMPLIANT - 1 violation(s) found

📈 SUMMARY:
   ✅ Compliant:       19
   ❌ Non-Compliant:   1
   📋 Total Checked:   20
   📊 Compliance Rate: 95.0%

🚨 VIOLATIONS FOUND (1)

1. Regulation (EU) 2016/679 (General Data Protection Regulation) — Principles relating to processing of personal data (Article 5)
   ID: GDPR Article 5(1)-(2)
   Issue: GDPR Art. 5(1)(d) requires personal data to be accurate and kept up to date, and that controllers take every reasonable step to erase/rectify inaccurate data without delay. The document includes a bro
   Evidence: "“We do not warrant that any information provided by us is accurate, complete, or useful...”..."
   Explanation: The GDPR accuracy principle (Art. 5(1)(d)) is a mandatory obligation on controllers to ensure personal data accuracy and to rectify/erase inaccurate d...



In [20]:
# Save the full report
from CCM import export_report_to_json

export_report_to_json(report, "compliance_report.json")

💾 Report saved to 'compliance_report.json'


---

## Step 4: Analyze Results

In [21]:
# Display summary statistics
summary = report["summary"]

print("📊 ARCCS COMPLIANCE SUMMARY")
print("=" * 50)
print(f"   ✅ Compliant:       {summary['compliant']:3d} ({summary['compliant']/summary['total']*100:.1f}%)")
print(f"   ❌ Non-Compliant:   {summary['non_compliant']:3d} ({summary['non_compliant']/summary['total']*100:.1f}%)")
print(f"   📋 Total Checked:   {summary['total']:3d}")
print(f"   📊 Compliance Rate: {summary['compliance_rate']}%")
print("=" * 50)

if summary['non_compliant'] == 0:
    print("\n🎉 CONGRATULATIONS! No compliance violations found!")
else:
    print(f"\n⚠️  ATTENTION: {summary['non_compliant']} violation(s) need to be addressed!")

📊 ARCCS COMPLIANCE SUMMARY
   ✅ Compliant:        19 (95.0%)
   ❌ Non-Compliant:     1 (5.0%)
   📋 Total Checked:    20
   📊 Compliance Rate: 95.0%

⚠️  ATTENTION: 1 violation(s) need to be addressed!


In [22]:
# Display violations in detail
if report['violations']:
    print("🚨 VIOLATIONS FOUND")
    print("=" * 70)
    
    for i, violation in enumerate(report['violations'], 1):
        print(f"\n{'─'*70}")
        print(f"❌ VIOLATION #{i}")
        print(f"{'─'*70}")
        print(f"📜 Regulation: {violation.get('regulation_name', 'Unknown')}")
        print(f"🔖 ID: {violation.get('regulation_id', 'N/A')}")
        
        if violation.get('contradiction_details'):
            print(f"\n📝 Issue:")
            print(f"   {violation['contradiction_details'][:300]}...")
        
        if violation.get('evidence'):
            print(f"\n📄 Evidence from document:")
            evidence = violation['evidence']
            print(f"   \"{evidence[:200]}...\"" if len(evidence) > 200 else f"   \"{evidence}\"")
        
        if violation.get('explanation'):
            print(f"\n💡 Explanation:")
            print(f"   {violation['explanation'][:200]}...")
    
    print(f"\n{'='*70}")
else:
    print("✅ No violations to display!")

🚨 VIOLATIONS FOUND

──────────────────────────────────────────────────────────────────────
❌ VIOLATION #1
──────────────────────────────────────────────────────────────────────
📜 Regulation: Regulation (EU) 2016/679 (General Data Protection Regulation) — Principles relating to processing of personal data (Article 5)
🔖 ID: GDPR Article 5(1)-(2)

📝 Issue:
   GDPR Art. 5(1)(d) requires personal data to be accurate and kept up to date, and that controllers take every reasonable step to erase/rectify inaccurate data without delay. The document includes a broad disclaimer that WhatsApp does not warrant accuracy of information it provides, which conflicts wi...

📄 Evidence from document:
   "“We do not warrant that any information provided by us is accurate, complete, or useful...”"

💡 Explanation:
   The GDPR accuracy principle (Art. 5(1)(d)) is a mandatory obligation on controllers to ensure personal data accuracy and to rectify/erase inaccurate data without delay. The Terms contain an expl

In [23]:
# Display compliant regulations
compliant_results = [r for r in report['detailed_results'] if r.get('compliance_status') == 'COMPLIANT']

print(f"✅ COMPLIANT REGULATIONS ({len(compliant_results)})")
print("=" * 70)

for i, result in enumerate(compliant_results[:10], 1):
    print(f"   {i}. {result.get('regulation_name', 'Unknown')[:60]}")

if len(compliant_results) > 10:
    print(f"   ... and {len(compliant_results) - 10} more")

✅ COMPLIANT REGULATIONS (19)
   1. General Data Protection Regulation (GDPR) – Material scope
   2. Regulation (EC) No 45/2001 (EU institutions personal data pr
   3. Directive 2000/31/EC (E-Commerce Directive) – Intermediary s
   4. Regulation (EU) 2016/679 (General Data Protection Regulation
   5. Regulation (EU) 2016/679 (General Data Protection Regulation
   6. Directive (EU) 2015/1535 of the European Parliament and of t
   7. Regulation (EU) 2016/679 (General Data Protection Regulation
   8. Regulation (EU) 2016/679 (General Data Protection Regulation
   9. Regulation (EU) 2016/679 (General Data Protection Regulation
   10. Regulation (EU) 2016/679 (General Data Protection Regulation
   ... and 9 more


---

## 📋 ARCCS Summary

This demo demonstrated the ARCCS system with its two modules:

### RPEM (Regulatory Processing and Extraction Module)
- Loaded and parsed a regulation PDF (GDPR)
- Extracted structured regulation data using AI
- Filtered regulations by quality

### CCM (Compliance Classification Module)
- Checked Terms of Service against regulations
- Detected contradictions (violations)
- Generated compliance report with explanations

### Output Files:
- `extracted_regulations.json` - Extracted regulations from RPEM
- `compliance_report.json` - Compliance analysis from CCM

### Labels Used:
- **COMPLIANT** - No contradiction found between document and regulation
- **NON_COMPLIANT** - Contradiction detected (document says X, regulation requires Y)

---

## 🔧 Quick Reference

### Check a Different Document

```python
from CCM import check_all_regulations, generate_compliance_report, print_detailed_report

# Load a different Terms of Service
with open("path/to/your/terms.txt", "r") as f:
    new_terms = f.read()

# Run compliance check
results = check_all_regulations(
    regulations=kept_regulations,
    proposal_chunk=new_terms,
    model="gpt-5.2"
)

# Generate report
report = generate_compliance_report(results)
print_detailed_report(report)
```

### Process a New Regulation Document

```python
from RPEM import process_regulation_document

# Complete pipeline
output = process_regulation_document(
    pdf_path="path/to/regulation.pdf",
    output_path="regulations.json",
    model="gpt-4.1",
    min_quality_score=40
)

regulations = output["filtered_regulations"]
```

### Load Saved Regulations

```python
with open("extracted_regulations.json", "r") as f:
    data = json.load(f)
    saved_regulations = data["regulations"]

print(f"Loaded {len(saved_regulations)} regulations")
```